<a href="https://colab.research.google.com/github/Marine27/TER/blob/master/PatternMining/Process_for_structure_pm(colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mise en forme des donnees pour le *Pattern Mining*

### Imports

In [0]:
import pandas as pd  
import numpy as np
import warnings

### Recuperation et preparation des donnnees

In [4]:
#Chargement du jeu de donnees
data_child = pd.read_csv('data_madeleine_final.csv',
                           sep = '\t',
                           encoding = 'utf-8',
                           index_col = False)

# Supression de la colonne qui duplique l'index
data_child = data_child.drop(columns = 'Unnamed: 0') 

# Recuperation de la partie phonetique des intervention de l'enfant
data_child = data_child.where(data_child['type']=='pho').dropna(subset=['type'])

# Conservation des deux colonnes avec des informations pertinantes 
data_child = data_child[['age','contenu']]

# Creer un indexage a partir de cet ordre
data_child['index']=[ i for i in range(len(data_child))]
data_child= data_child.set_index('index')

# Creation d'un numpy utile par la suite pour les boucles
ages = data_child.age.unique()

# Indexage par l'age de l'enfant
age = data_child['age']

l=[i for i in range(len(age))]
k=[age.values]
k.append(l)
tuples = list(zip(*k))

index = pd.MultiIndex.from_tuples(tuples, names=['age', 'index'])
data_child.set_index(index,inplace=True)

# Supression de la colonne age car present en index
data_child=data_child[['contenu']]

data_child.head()

contenu
age     index        
1_00_05 0          eː
        1          bɛ
        2        vœga
        3       gatœ̃
        4        kaka

### Demarches pour la division de chaque enregistrement en 4 parties

In [0]:
# Sépare en mots au lieu de phrases
mots = {}
for i in list(ages):
    # recupere toutes les phrases a la i eme date
    phrases = data_child['contenu'].loc[i].values
    # creer une variable texte avec toutes les phrases
    phrase = ' '.join(phrases)
    # sépare en mot
    mots[i] = phrase.split()
    
# se parcourt avec un age en parametre
#mots["1_00_05"]

In [6]:
# Sépare en 4 "enfants" chaque enregistrement
enregistrement = {}
sizes = []
for i in list(ages):
    # recupere tout les mots a la i eme date
    #data = data_child['contenu'].loc[i].values
    data = mots[i]
    # separe les mots en 4 groupes ( 3 goupes de meme dim et le dernier avec le reste)
    if len(data)<4:
        warnings.warn("\nAttention un l'enregistrement "+ i + " contient moins de 4 mots")
    s = len(data)//4
    sizes.append(s)
    enfant1 = data[:s]
    enfant2 = data[s:2*s]
    enfant3 = data[2*s:3*s]
    enfant4 = data[3*s:]
    # creer une variable texte de chaque goupe
    #enfant1 = ' '.join(enfant1)
    #enfant2 = ' '.join(enfant2)
    #enfant3 = ' '.join(enfant3)
    #enfant4 = ' '.join(enfant4)
    
    enfant1 = tuple(enfant1)
    enfant2 = tuple(enfant2)
    enfant3 = tuple(enfant3)
    enfant4 = tuple(enfant4)
    
    # sépare en 4 enfants
    enregistrement[i] = [enfant1, enfant2, enfant3, enfant4]
    
# se parcourt avec un age en parametre
#enregistrement["1_00_05"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: 
Attention un l'enregistrement 6_11_27 contient moins de 4 mots
  if __name__ == '__main__':


In [7]:
# Pour avoir une idée du nombre de mots par enregistrement après répartition
pd.DataFrame(sizes[:-1]).describe().T

,count,mean,std,min,25%,50%,75%,max
0,34.0,395.588235,285.178744,15.0,126.75,364.0,646.25,1101.0


In [8]:
# Crée une DataFrame de toutes les donnees
tab_4groupes = []
#sizes = []
for i in list(ages):
    t = enregistrement[i]
    #s = len(t)
    tab_4groupes.append(t)
    #sizes.append(s)
    
tab_madeleine = pd.DataFrame(tab_4groupes)

etiquettes = ["madeleine_enfant1",
              "madeleine_enfant2", 
              "madeleine_enfant3", 
              "madeleine_enfant4"]

tab_madeleine.columns = etiquettes

tab_madeleine.head()
# tab_madeleine.loc[1] recherche par numero de l'enregistrement
# tab_madeleine[1] recherche par numero de l'enfant

,madeleine_enfant1,madeleine_enfant2,madeleine_enfant3,madeleine_enfant4
0,"(eː, bɛ, vœga, gatœ̃, kaka, dœjøkɛka, m, taka,...","(gla, ga, ga, gøga, gøga, ga, gaga, a, deda, œ...","(gaga, gaga, gagaga, ga, mœmœmœmœ, ala, adla, ...","(œmøː, œmøː, gaga, gagaga, daɥy, aga, œm, œm, ..."
1,"(ja, aljɛla, jaka, meja, agø, ga, beːja, ʃa, o...","(ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, aja, ja, ...","(ʃa, ejaː, edjadjajaaa, edahahaoga, u, ajne, e...","(ada, aja, gɛʒeʒa, ajəda, dedaː, da, ega, ja, ..."
2,"(je, ʃidɔm, øʃi, alapa, ʃː, ʃ, a, ʃa, øː, aaː,...","(bɛ, y, øː, ʃ, mememebuja, eaa, iɲa, ia, ʃɛ̃, ...","(ʃa, ʒy, aːaːɛ, ɛɛ, sə, wauːua, a, χ, wa, waa,...","(bidun, bida, bjɛtna, bebøː, dakula, daguba, m..."
3,"(evøpuʃa, gegedɔ̃, dɔ, ty, hɛsys, ejə, ʃa, gøt...","(mama, mama, bekaliʃaʃa, øtatɛ, mama, løʃa, pe...","(ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃːa, səba, ʃa, ʃa...","(lelɛ, lala, lala, uu, iː, jejejejejee, eː, mː..."
4,"(i, lɛ, vwa, a, aː, ti, vwa, aː, ty, pø, t, as...","(wikɔ, ejdøwa, ɛʃa, pøpa, ɛøgødididitetɛe, bøb...","(wœ, wa, wawa, pwapwa, awœ, œœ, ba, m, jeje, e...","(jije, ini, œː, i, ʃi, uba, veve, tilele, nine..."


### Addaptation de la structure

In [0]:
# transactions = (('a', 'b', 'c'), ('b'), ('a'), ('a', 'c', 'd'), ('b', 'c'), ('b', 'c'))
# doit etre de cette forme:
# madeleine_enfant1 = transactions
# un enregistrement = ('a', 'b', 'c')
# un mot = 'a'

In [0]:
madeleine_enfant1 = tuple(tab_madeleine["madeleine_enfant1"][:-1])
madeleine_enfant2 = tuple(tab_madeleine["madeleine_enfant2"][:-1])
madeleine_enfant3 = tuple(tab_madeleine["madeleine_enfant3"][:-1])
madeleine_enfant4 = tuple(tab_madeleine["madeleine_enfant4"][:])

In [11]:
print(type(madeleine_enfant1))
#print(madeleine_enfant1)

<class 'tuple'>


### PATTERN MINING

In [12]:
!pip install pymining
from pymining import itemmining
from pymining import assocrules
from pymining import seqmining

### support min = 20

In [0]:
min_supp = 25
relim_input_1 = itemmining.get_relim_input(madeleine_enfant1)
relim_input_2 = itemmining.get_relim_input(madeleine_enfant2)
relim_input_3 = itemmining.get_relim_input(madeleine_enfant3)
relim_input_4 = itemmining.get_relim_input(madeleine_enfant4)

item_sets_1 = itemmining.relim(relim_input_1, min_support=min_supp) 
item_sets_2 = itemmining.relim(relim_input_2, min_support=min_supp)
item_sets_3 = itemmining.relim(relim_input_3, min_support=min_supp)
item_sets_4 = itemmining.relim(relim_input_4, min_support=min_supp)



In [14]:
print(len(item_sets_1))
print(len(item_sets_2))
print(len(item_sets_3))
print(len(item_sets_4))

20
20
14
18


In [0]:
#item_sets_1

In [0]:
#item_sets_2

In [0]:
#item_sets_3

In [0]:
#item_sets_4

In [0]:
rules_1 = assocrules.mine_assoc_rules(item_sets_1, min_support=min_supp, min_confidence=0.8)
rules_2 = assocrules.mine_assoc_rules(item_sets_2, min_support=min_supp, min_confidence=0.8)
rules_3 = assocrules.mine_assoc_rules(item_sets_3, min_support=min_supp, min_confidence=0.8)
rules_4 = assocrules.mine_assoc_rules(item_sets_4, min_support=min_supp, min_confidence=0.8)

In [20]:
print(len(rules_1))
print(len(rules_2))
print(len(rules_3))
print(len(rules_4))

29
52
26
50


In [0]:
#rules_1

In [0]:
#rules_2

In [0]:
#rules_3

In [0]:
#rules_4

In [0]:
seqs_1 = madeleine_enfant1
seqs_2 = madeleine_enfant2
seqs_3 = madeleine_enfant3
seqs_4 = madeleine_enfant4

freq_seqs_1 = seqmining.freq_seq_enum(seqs_1, 20)
freq_seqs_2 = seqmining.freq_seq_enum(seqs_2, 20)
freq_seqs_3 = seqmining.freq_seq_enum(seqs_3, 20)
freq_seqs_4 = seqmining.freq_seq_enum(seqs_4, 20)

#for seq in freq_seqs:
#    print (seq)

In [26]:
print(len(freq_seqs_1))
print(len(freq_seqs_2))
print(len(freq_seqs_3))
print(len(freq_seqs_4))

4800
5373
10290
5640


In [0]:
for seq in freq_seqs_1:
    print (seq)

In [0]:
for seq in freq_seqs_2:
    print (seq)

In [0]:
for seq in freq_seqs_3:
    print (seq)

In [0]:
for seq in freq_seqs_4:
    print (seq)